In [1]:
%run convention.ipynb

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Custom Layer

## Custom layer without weights

<p class = 'highlight'>use <code>keras.layers.Lambda(function)</code></p>

In [24]:
import keras
from keras.layers import Lambda, Layer, Dense
from sklearn.datasets import load_iris
from keras.models import Sequential
import tensorflow as tf

In [5]:
sigmoid = Lambda(lambda z: 1 / (1 + tf.exp(-z)))

## Custom layer with weights:

<p class = 'highlight'>Create a subclass of <code>keras.layers.Layer</code>:</p>

In [35]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation = None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name = 'kernel', shape = [batch_input_shape[-1], self.units], initializer = 'glorot_normal')
        self.bias = self.add_weight(name = 'bias', shape = [self.units], initializer = 'zeros')
        super().build(batch_input_shape)
    def call(self, X):
        return self.kernel @ X + self.bias
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units':self.units, 'activation': keras.activations.serialize(self.activation)}

In [37]:
class MyDense1(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [50]:
X, y = load_iris(return_X_y=True)
# model = Sequential([
#     MyDense(10, activation = 'relu', input_shape = (4,)),
#     MyDense(3, activation = 'softmax')
# ])
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=(4,)),
    MyDense(1)
])
model.summary()

ValueError: Dimensions must be equal, but are 4 and 30 for 'my_dense_22/add' (op: 'AddV2') with input shapes: [4,4], [30].

In [45]:

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(X, y, epochs = 30)

AttributeError: 'Sequential' object has no attribute '_feed_input_names'

In [44]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

<img src = 'custom_layer.png'/>